Import packages


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm
import joblib

Import dataset (We will use the smaller one first)

In [21]:
data = pd.read_csv("../data/50k-sample-song-features.csv", index_col = 0)
data

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,11NFxVnEwPDz4LjIyKazxV,Allein sein,['Mario Novembre'],0.690,0.522,9,-8.108,1,0.0541,0.13000,0.000000,0.1100,0.511,155.048,audio_features,spotify:track:11NFxVnEwPDz4LjIyKazxV,https://api.spotify.com/v1/tracks/11NFxVnEwPDz...,https://api.spotify.com/v1/audio-analysis/11NF...,180387,4
1,5QEqzFV7R8KSSCJa37tfvs,Leere Hände (feat. Sido & Samra),"['SANTOS', 'Sido', 'Samra']",0.683,0.699,8,-4.519,0,0.2180,0.25200,0.000000,0.2180,0.315,160.102,audio_features,spotify:track:5QEqzFV7R8KSSCJa37tfvs,https://api.spotify.com/v1/tracks/5QEqzFV7R8KS...,https://api.spotify.com/v1/audio-analysis/5QEq...,169191,4
2,0b18g3G5spr4ZCkz7Y6Q0Q,Rasputin,"['Majestic', 'Boney M.']",0.758,0.913,6,-2.926,0,0.1130,0.00364,0.000069,0.1930,0.658,128.040,audio_features,spotify:track:0b18g3G5spr4ZCkz7Y6Q0Q,https://api.spotify.com/v1/tracks/0b18g3G5spr4...,https://api.spotify.com/v1/audio-analysis/0b18...,186210,4
3,58GxgFEG5uHc9oMhofuuIP,Drei Uhr Nachts,"['Mark Forster', 'LEA']",0.663,0.673,8,-7.031,1,0.1190,0.06010,0.000014,0.1910,0.759,164.817,audio_features,spotify:track:58GxgFEG5uHc9oMhofuuIP,https://api.spotify.com/v1/tracks/58GxgFEG5uHc...,https://api.spotify.com/v1/audio-analysis/58Gx...,161001,4
4,0ZqL6UhrqydZE4DaXvCMeT,Liege wieder wach,['Jamule'],0.743,0.599,11,-9.125,1,0.0614,0.33900,0.000003,0.0897,0.436,82.017,audio_features,spotify:track:0ZqL6UhrqydZE4DaXvCMeT,https://api.spotify.com/v1/tracks/0ZqL6UhrqydZ...,https://api.spotify.com/v1/audio-analysis/0ZqL...,176251,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49519,4RwRGbPsifdX7x9NuX8oMv,Life Is Beautiful,['Sanjith Hegde'],0.585,0.685,4,-7.752,1,0.0359,0.09380,0.000024,0.1600,0.379,119.996,audio_features,spotify:track:4RwRGbPsifdX7x9NuX8oMv,https://api.spotify.com/v1/tracks/4RwRGbPsifdX...,https://api.spotify.com/v1/audio-analysis/4RwR...,174000,4
49520,7Diocah252ihnpOwC4KCny,Don't Worry,['All Ok'],0.630,0.688,1,-5.720,0,0.0763,0.39700,0.082800,0.1170,0.629,87.902,audio_features,spotify:track:7Diocah252ihnpOwC4KCny,https://api.spotify.com/v1/tracks/7Diocah252ih...,https://api.spotify.com/v1/audio-analysis/7Dio...,214091,4
49521,0dQCsUtyN9RDtF0LDKnyQL,Sidila Bharava,"['Santhosh', 'Sachin', 'Puneeth Rajkumar', 'Mo...",0.533,0.909,10,-6.106,0,0.0663,0.32000,0.001290,0.6300,0.238,127.983,audio_features,spotify:track:0dQCsUtyN9RDtF0LDKnyQL,https://api.spotify.com/v1/tracks/0dQCsUtyN9RD...,https://api.spotify.com/v1/audio-analysis/0dQC...,210545,4
49522,6AVyr9vQGflNFYZICTXIf0,Tajaa Samachara - Male,['Jithin Raj'],0.622,0.769,0,-3.822,0,0.0329,0.17100,0.000000,0.2280,0.594,124.957,audio_features,spotify:track:6AVyr9vQGflNFYZICTXIf0,https://api.spotify.com/v1/tracks/6AVyr9vQGflN...,https://api.spotify.com/v1/audio-analysis/6AVy...,289894,4


We extract the numerical features we want to use. 

In [35]:
X = data.loc[:,'danceability':'valence'].drop(['mode'], axis = 1)

In [36]:
X.columns

Index(['danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence'],
      dtype='object')

In [37]:
scaler = StandardScaler().fit(X)

we save it for later use 

In [38]:
joblib.dump(scaler, '../data/model/std_scaler.bin', compress=True)

['../data/model/std_scaler.bin']

In [39]:
X_prep = scaler.transform(X)

In [40]:
X_prep_df = pd.DataFrame(X_prep, columns=X.columns)

In [41]:
X_prep_df

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,0.646217,-0.009625,1.065826,0.349455,-0.267625,-0.830944,-0.572029,-0.423145,0.163290
1,0.609356,0.633849,0.784109,0.870324,1.681179,-0.493100,-0.572029,0.387304,-0.565042
2,1.004294,1.411834,0.220674,1.101515,0.432708,-1.180862,-0.571828,0.199700,0.709539
3,0.504039,0.539327,0.784109,0.505759,0.504049,-1.024512,-0.571990,0.184692,1.084853
4,0.925306,0.270304,1.629261,0.201859,-0.180827,-0.252178,-0.572021,-0.575480,-0.115408
...,...,...,...,...,...,...,...,...,...
49519,0.093304,0.582953,-0.342761,0.401121,-0.484027,-0.931190,-0.571958,-0.047937,-0.327219
49520,0.330266,0.593859,-1.187913,0.696024,-0.003663,-0.091563,-0.330479,-0.370616,0.601775
49521,-0.180520,1.397292,1.347543,0.640004,-0.122565,-0.304793,-0.568266,3.479021,-0.851172
49522,0.288140,0.888330,-1.469630,0.971479,-0.519698,-0.717406,-0.572029,0.462346,0.471716


In [51]:
kmeans = KMeans(n_clusters=50, random_state=666)
kmeans.fit(X_prep_df)

KMeans(n_clusters=50, random_state=666)

and we save it again

In [52]:
joblib.dump(kmeans,"../data/model/kmeans_model.sav")

['../data/model/kmeans_model.sav']

In [53]:
kmeans.cluster_centers_

array([[ 2.37222666e-01,  1.86376398e-01, -1.04559484e+00,
         3.57755390e-01, -2.64714861e-01,  5.45327423e-01,
        -4.97821441e-01, -2.25687259e-01,  8.97108329e-01],
       [-1.50614846e+00, -1.27122361e+00, -9.34646435e-01,
        -1.10284608e+00, -4.33479253e-01,  1.29124797e+00,
         1.84784837e+00, -2.60809872e-01, -1.28337703e+00],
       [ 8.95419041e-01,  5.60604750e-01, -9.77871720e-01,
         5.80203970e-01,  1.77002903e+00, -5.45311261e-01,
        -5.34318394e-01, -3.58872127e-01,  5.57866980e-01],
       [-8.25949923e-01,  1.26129561e+00,  8.16784341e-01,
         7.87654876e-01,  2.06300607e-01, -1.10414773e+00,
        -4.59972960e-01,  7.90690545e-02, -5.81816344e-01],
       [ 2.90416879e-01, -6.41638070e-01,  1.02988091e+00,
         2.66368615e-03, -3.71032779e-01,  8.59762167e-01,
        -5.11634241e-01, -3.08597763e-01, -3.52757463e-01],
       [ 1.75978831e-01, -5.34738537e-01, -1.02556051e+00,
         8.67157793e-02, -4.03306536e-01,  7.553128

In [54]:
kmeans.inertia_

92154.025812592

In [55]:
clusters = kmeans.predict(X_prep)
len(clusters)

49524

In [56]:
pd.Series(clusters).value_counts().sort_index()

0     1255
1     1109
2     1073
3      958
4     1590
5     1469
6      807
7     1356
8     1526
9     1099
10     647
11    1521
12    1780
13    1007
14     212
15     457
16     769
17    1302
18     103
19    1042
20     571
21     327
22    1242
23    1364
24     836
25    1306
26     854
27    1466
28     577
29     755
30     732
31    1426
32    1257
33     698
34    1394
35     427
36     690
37     767
38     290
39    1614
40    1195
41    1447
42     415
43    1281
44     980
45    1242
46     946
47     522
48     513
49    1308
dtype: int64

In [57]:
X_df = pd.DataFrame(X)
X_df['cluster'] = clusters
X_df

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,cluster
0,0.690,0.522,9,-8.108,0.0541,0.13000,0.000000,0.1100,0.511,49
1,0.683,0.699,8,-4.519,0.2180,0.25200,0.000000,0.2180,0.315,22
2,0.758,0.913,6,-2.926,0.1130,0.00364,0.000069,0.1930,0.658,12
3,0.663,0.673,8,-7.031,0.1190,0.06010,0.000014,0.1910,0.759,25
4,0.743,0.599,11,-9.125,0.0614,0.33900,0.000003,0.0897,0.436,49
...,...,...,...,...,...,...,...,...,...,...
49519,0.585,0.685,4,-7.752,0.0359,0.09380,0.000024,0.1600,0.379,8
49520,0.630,0.688,1,-5.720,0.0763,0.39700,0.082800,0.1170,0.629,0
49521,0.533,0.909,10,-6.106,0.0663,0.32000,0.001290,0.6300,0.238,16
49522,0.622,0.769,0,-3.822,0.0329,0.17100,0.000000,0.2280,0.594,7


In [58]:
#We do an id-cluster dataframe
id_cluster = pd.concat([X_df['cluster'],data[['artists','name','id']]],axis = 1)


In [59]:
id_cluster.to_csv("../data/model/cluster-id-dataframe.csv")

In [ ]:
# I want to iterate over a range of n_clusters and for every value, I want to return the inertia
def get_kmeans_inertia_varying_cluster_n(n_clusters):
    
    # setup the model
    kmeans = KMeans(n_clusters=n_clusters,
                    random_state=666,
                    n_init=3,
                    #algorithm='elkan',
                   )
    # train the model
    kmeans.fit(X_prep_df)
    
    # return the resulting inertia
    return kmeans.inertia_

# Plot for a range of cluster numbers
import matplotlib.pyplot as plt

cluster_range = range(2,30)

plt.plot(cluster_range,
         [get_kmeans_inertia_varying_cluster_n(c_number) for c_number in cluster_range],
         marker="o",
         ms=10,
        )
plt.xlabel('Cluster Number')
plt.ylabel('inertia')

In [ ]:
# I want to iterate over a range of mx_iter and for every value, I want to return the inertia
def get_kmeans_ineratia_varying_max_iter(max_iter):
    kmeans = KMeans(n_clusters=5,
                    random_state=666,
                    n_init=3,
                    algorithm='elkan',
                    max_iter=max_iter,
                   )
    kmeans.fit(X_prep_df)

    return kmeans.inertia_

max_iter_list = [1, 5, 10, 20, 30, 40, 50, 100]

plt.plot(max_iter_list,
         [get_kmeans_ineratia_varying_max_iter(x) for x in max_iter_list],
        )
plt.xlabel('Max iter')
plt.ylabel('inertia')

In [ ]:
from sklearn.metrics import silhouette_score

K = range(2, 30)

silhouettes = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                   random_state=666)
    kmeans.fit(X_prep)
    silhouettes.append(silhouette_score(X_prep, kmeans.predict(X_prep)))


In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(16,8))
plt.plot(K, silhouettes, 'bo-')
plt.xlabel('k (number of clusters)')
plt.ylabel('silhouette score')

In [ ]:
K = range(10, 100)

silhouettes = []

for k in tqdm(K):
    kmeans = KMeans(n_clusters=k,
                   random_state=666)
    kmeans.fit(X_prep)
    silhouettes.append(silhouette_score(X_prep, kmeans.predict(X_prep)))

plt.figure(figsize=(16,8))
plt.plot(K, silhouettes, 'bo-')
plt.xlabel('k (number of clusters)')
plt.ylabel('silhouette score')

We'll do 24 clusters! 

In [ ]:
km = KMeans(n_clusters=24, random_state=666)
visualizer = SilhouetteVisualizer(km, colors='yellowbrick')
visualizer.fit(X_prep)